In [ ]:
"""
IMPORTS
"""

import numpy as np
import pandas as pd

In [ ]:
"""
FUNCIONS
"""

def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

def clean_dataset(dataset: pd.core.frame.DataFrame, type = 'default') -> pd.core.frame.DataFrame:
    """
    Funció que processa els valors "NULLS" d'un dataset aplicant l'estratègia o tractament especificat
    :param dataset: DataFrame amb la informació que es vol filtrar
    :param type: Indica el tipus de tractament dels  "NaN"
    :return: DataFrame aplicant el mètode especificat a totes les línies amb valors "NaN"
    """
    if type == 'default':
        # Deletes all rows with missing values
        return dataset.dropna()
    elif type == 'backfill':
        # Applies pandas method of backfilling
        return dataset.fillna(method='backfill', axis=1)
    elif type == 'mean':
        # Replaces missing values with the mean of the column
        return dataset.fillna(dataset.mean())
    else:
        print("::-> ERROR : clean_dataset - " + str(type) + " is not a valid option...")






In [ ]:
# PREPROCESSING

from sklearn.preprocessing import OneHotEncoder

dataset = load_dataset('pokemon.csv')

# Feature Selection : Elimianció atributs que no aporten informació
dataset = dataset.drop(['japanese_name', 'name', 'pokedex_number'], axis=1)

# Eliminació de classification, doncs és un atribut que expressa si és legendary o no i aquest és el nostre objectiu a determinar
dataset = dataset.drop(['classfication'], axis=1)

# Eliminació atribut que complica el nostre model
# Null en aquesta linia convertir-ho en un tipus que no te
# dataset = dataset.drop(['type2'], axis=1)

# Codificació dels atributs categòrics amb mètode OneHotEncoder:

# Categoritzar Type1
# Definim encoder
encoder=OneHotEncoder(sparse=False)
# Creem variable auxiliar per guardar la informació del atribut type1
new_data_1 = pd.DataFrame (encoder.fit_transform(dataset[['type1']]))
# Assignem els noms a les columnes
new_data_1.columns = encoder.get_feature_names(['type1'])
# Eliminem la columna del dataset
dataset.drop(['type1'] ,axis=1, inplace=True)

# Categoritzar Type2
# Modifiquem nom de la columna type 2 a type 1 per tal de poder fer un merge posteriorment
# Utilitzarem el mateix procés que amb la variable type1
dataset = dataset.rename(columns = {'type2' : 'type1'})
new_data_2 = pd.DataFrame (encoder.fit_transform(dataset[['type1']]))
new_data_2.columns = encoder.get_feature_names(['type1'])
dataset.drop(['type1'] ,axis=1, inplace=True)

# Sumem les dues taules type
new_data_type = new_data_1 + new_data_2
# Afegim al final del dataset les taules type1 i type2 codificades amb OneHotEncoder
dataset = pd.concat([dataset, new_data_type], axis=1)

# Eliminem la columna type amb valors NULLS:
dataset = dataset.drop(['type1_nan'], axis=1)

# Error en linia 773. Parametres no valids
# dataset = dataset.drop(index=773, axis=0)
dataset['']

# Tractament valors NULLS -> Aplicarem mètode de la mitjana
dataset = clean_dataset(dataset, type='mean')



import ast
ab = dataset['abilities'].to_numpy()
a = []
for row in ab:
    row = ast.literal_eval(row)
    a.append(row)

a = np.array(a).flatten()

flat_list = [item for sublist in a for item in sublist]

abilitats_uniques = np.unique(flat_list).reshape(-1,1)

# Declarem encoder tipus OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
# Declarem Dataframe auxiliar amb les noves columnes
enc_df = pd.DataFrame(enc.fit_transform(abilitats_uniques).toarray(), columns=['abilities_' + str(i[0]) for i in abilitats_uniques])
# Ajuntem al dataframe original amb el dataframe auxiliar
dataset = dataset.join(enc_df)
dataset = dataset.drop(['abilities'], axis=1)



from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = dataset.values[:,34]
X = dataset.drop(['is_legendary'] ,axis=1, inplace=True)
X = dataset.values[:,:]

x_t, x_v, y_t, y_v = train_test_split(X, y, train_size=0.3)

param_grid = [
    {"C": np.logspace(-3,3,7), "penalty":["l1"], 'solver': ['saga'], 'multi_class': ['auto', 'ovr', 'multinomial']},
    {"C": np.logspace(-3,3,7), "penalty":["l2", "none"], 'solver': ['newton-cg','sag', 'lbfgs'], 'multi_class': ['auto', 'ovr', 'multinomial']}

]
log_reg = LogisticRegression()

grid_search = GridSearchCV(log_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', verbose=1)

grid_search.fit(X, y)

print(grid_search.best_params_)
print(grid_search.best_score_)
